In [26]:
!pip install azureml-core

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [43]:
from azureml.core import Workspace
from azureml.core import Environment
from azureml.core.environment import CondaDependencies
from azureml.core.compute import AksCompute, ComputeTarget
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AksWebservice
from azureml.core.model import Model
from azureml.core import Environment
from azureml.core.environment import CondaDependencies

In [44]:
ws = Workspace.from_config("/content/drive/MyDrive/Azure_Ml/config (2).json")

In [45]:
Model.register(workspace=ws,
               model_path='/content/drive/MyDrive/Colab Notebooks/Sales Forecasting/encoder.pkl', # local path
               model_name='Store_Sales_Pred_Encoder',
               tags={ 'Model_Name':'Time Series Encoder'},
               description='Stores Sales Forecasting Encoder'
               )
Model.register(workspace=ws,
               model_path='/content/drive/MyDrive/Colab Notebooks/Sales Forecasting/TS_Model.hdf5', # local path
               model_name='Store_Sales_Pred_Model',
               tags={ 'Model_Name':'Time Series Model'},
               properties={'MSLE': 0.1449},
               description='Stores Sales Forecasting Model'
               )

Registering model Store_Sales_Pred_Encoder
Registering model Store_Sales_Pred_Model


Model(workspace=Workspace.create(name='ML_Workspace', subscription_id='a903e32f-d960-49ea-90a4-dffb7864b432', resource_group='Archit_ML_Projects'), name=Store_Sales_Pred_Model, id=Store_Sales_Pred_Model:8, version=8, tags={'Model_Name': 'Time Series Model'}, properties={'MSLE': '0.1449'})

In [46]:
myenv = Environment(name="MyEnvironment")
myenv_dep = CondaDependencies.create(conda_packages=['scikit-learn', 'pip','tensorflow'],
                                     pip_packages=['azureml-defaults', 'pandas','category_encoders','dill','joblib == 1.2'])
myenv.python.conda_dependencies = myenv_dep
myenv.register(ws)

{
    "assetId": "azureml://locations/centralindia/workspaces/84913557-4d13-4678-8f5b-5e8dc46d04d4/environments/MyEnvironment/versions/8",
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:20220915.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "buildContext": null,
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "MyEnvironment",
  

In [47]:
cluster_name = 'AKS-Cluster-1'

if cluster_name not in ws.compute_targets:
    aks_config = AksCompute.provisioning_configuration(location='centralindia',
                                                       vm_size='STANDARD_D11_V2',
                                                       agent_count=1,
                                                       cluster_purpose='DevTest')
    production_cluster = ComputeTarget.create(ws, cluster_name, aks_config)
    production_cluster.wait_for_completion(show_output=True)
else:
    production_cluster = ws.compute_targets[cluster_name]


In [51]:
inference_config = InferenceConfig(
                                   entry_script="/content/drive/MyDrive/Colab Notebooks/Sales Forecasting/tms_pred.py",
                                   environment=myenv)

In [49]:
deploy_config = AksWebservice.deploy_configuration(cpu_cores = 1,
                                                   memory_gb = 1)

In [ ]:
!pip show joblib

Name: joblib
Version: 1.2.0
Summary: Lightweight pipelining with Python functions
Home-page: https://joblib.readthedocs.io
Author: Gael Varoquaux
Author-email: gael.varoquaux@normalesup.org
License: BSD
Location: /usr/local/lib/python3.7/dist-packages
Requires: 
Required-by: scikit-learn, nltk, librosa, imbalanced-learn


In [52]:
model1 = ws.models['Store_Sales_Pred_Encoder']
model2 = ws.models['Store_Sales_Pred_Model']

service = Model.deploy(workspace=ws,
                       name = 'salespred-service',
                       models = [model1,model2],
                       inference_config = inference_config,
                       deployment_config = deploy_config,
                       deployment_target = production_cluster,overwrite=True)
service.wait_for_deployment(show_output = True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoint-sdk-v2 /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration. 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  if __name__ == '__main__':


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-10-08 16:25:32+00:00 Creating Container Registry if not exists.
2022-10-08 16:25:32+00:00 Registering the environment.
2022-10-08 16:25:32+00:00 Use the existing image.
2022-10-08 16:25:34+00:00 Checking the status of deployment salespred-service..
2022-10-08 16:25:57+00:00 Checking the status of inference endpoint salespred-service.
Succeeded
AKS service creation operation finished, operation "Succeeded"
